In [ ]:
!pip install gdown
!pip install pystemmer
!pip install fuzzywuzzy
!pip install python-Levenshtein

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 559 kB 13.3 MB/s 
  Created wheel for pystemmer: filename=PyStemmer-2.0.1-cp37-cp37m-linux_x86_64.whl size=425930 sha256=ed2b3bc1b5301b8fd85cecad0f4b520bdb584865b822b6d211e27d5589026933
  Stored in directory: /root/.cache/pip/wheels/30/6d/40/0d17a498c5009922dbb3ddaca3d3652387ba94cc96142001f0
Successfully built pystemmer
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 50 kB 5.7 MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149873 sha256=54734084759a4f128e61950be8b47278105f9bca33ecedbc3e1aa50f0fbb13e1

In [ ]:
import Stemmer, os, pickle
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [ ]:
# path = os.path.join(os.getcwd())#, "files")
# file_list = [os.path.join(path,i) for i in os.listdir(path) if os.path.isfile(i)]
# print(file_list)

In [ ]:
!gdown --id 1DWazm5rnGBb7zQmbRhgCYDzYNwlmLyvQ
!gdown --id 15QnJJeUAVrLG1x84u2CAnf6fNkXMew98

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1DWazm5rnGBb7zQmbRhgCYDzYNwlmLyvQ
To: /content/datasetFile.txt
100% 4.03M/4.03M [00:00<00:00, 152MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=15QnJJeUAVrLG1x84u2CAnf6fNkXMew98
To: /content/combined.pickle
100% 452k/452k [00:00<00:00, 53.7MB/s]


In [ ]:
def Merge(dict1, dict2):
    return(dict2.update(dict1))

In [ ]:
# res = {}
# for i in file_list:
#   if i == "/content/datasetFile.txt":
#     continue
#   with open(i, "rb") as handle:
#     print(i)
#     b = pickle.load(handle)
#     Merge(b, res)

res = {}
with open('combined.pickle','rb') as handle:
  res = pickle.load(handle)

In [ ]:
#res

In [ ]:
rootwords = []
chdf = open('datasetFile.txt')
for r in chdf.readlines():
        rootwords.append(r.strip())

In [ ]:
def fuzzymatch(string, master = rootwords):
  score = 0
  arr = []
  d = {}
  for (i,j) in process.extract(string, master, limit = 10):
    ratio = fuzz.partial_ratio(i,string)
    if ratio >= 85:
      score += 1
      arr.append(i)
      d[i] = ratio
  return (score, arr, d)
  

In [ ]:
stemmer = Stemmer.Stemmer('tamil')

In [ ]:
result = {}
for i in res:
  result[i] = {
      "word" : i,
      "pystemmer" : stemmer.stemWord(i),
      "wordsplitting": list(res[i]['root_score'].keys())
  }

In [ ]:
import pandas as pd
df = pd.DataFrame(result)
df = df.transpose()
#df = df.drop(columns=df.columns[0], axis=1, inplace=True)

In [ ]:
def part_res_func(row):
  words = [row["pystemmer"]] + row["wordsplitting"]
  word = row["word"]
  d = {}
  for (i,j) in process.extract(word, words, limit = 10):
    ratio = fuzz.partial_ratio(i,word)
    d[i] = ratio
  return d

In [ ]:
def res_func(row):
  words = [row["pystemmer"]] + row["wordsplitting"]
  word = row["word"]
  d = {}
  for (i,j) in process.extract(word, words, limit = 10):
    ratio = fuzz.ratio(i,word)
    d[i] = ratio
  return d

In [ ]:
df["part_res"] = df.apply(part_res_func, axis=1)
df["res"] = df.apply(res_func, axis=1)

In [ ]:
def part_mean_func(row):
  res = row['part_res']
  # print(res)
  if res == {}:
    return 0
  score = 0
  for i in res:
    score += res[i]
  return score/len(res)

def mean_func(row):
  res = row['res']
  # print(res)
  if res == {}:
    return 0
  score = 0
  for i in res:
    score += res[i]
  return score/len(res)

In [ ]:
df["partial_mean"] = df.apply(part_mean_func, axis=1)
df["mean"] = df.apply(mean_func, axis=1)

In [ ]:
df.sample(200)

,word,pystemmer,wordsplitting,part_res,res,partial_mean,mean
தெய்வதூஷணம்,தெய்வதூஷணம்,தெய்வதூஷணம்,[தெய்வதூஷணம்],{'தெய்வதூஷணம்': 100},{'தெய்வதூஷணம்': 100},100.0,100.0
படர்ந்து,படர்ந்து,படர்,[],{'படர்': 100},{'படர்': 67},100.0,67.0
நானாளைமாறன்,நானாளைமாறன்,நானாளைமாறன்,[நானாளைமாறன்],{'நானாளைமாறன்': 100},{'நானாளைமாறன்': 100},100.0,100.0
சந்தித்தான்,சந்தித்தான்,சந்தி,[],{'சந்தி': 100},{'சந்தி': 62},100.0,62.0
எஞ்சியிருக்கும்,எஞ்சியிருக்கும்,சியிர்,[],{'சியிர்': 83},{'சியிர்': 57},83.0,57.0
...,...,...,...,...,...,...,...
பொன்னை,பொன்னை,பொன்,[],{'பொன்': 100},{'பொன்': 80},100.0,80.0
தன்னிலை,தன்னிலை,தன்னில்,[தன்னிலை],"{'தன்னில்': 86, 'தன்னிலை': 100}","{'தன்னில்': 86, 'தன்னிலை': 100}",93.0,93.0
ஞகம்,ஞகம்,ஞகம்,"[ஞகம், கம்]","{'ஞகம்': 100, 'கம்': 100}","{'ஞகம்': 100, 'கம்': 86}",100.0,93.0
ஊர்ந்து,ஊர்ந்து,ஊர்,[],{'ஊர்': 100},{'ஊர்': 60},100.0,60.0


In [ ]:
df.describe()

,partial_mean,mean
count,7008.000000,7008.000000
mean,96.759933,77.977171
std,7.413426,17.678337
min,0.000000,0.000000
25%,95.500000,67.000000
50%,100.000000,80.000000
75%,100.000000,93.000000
max,100.000000,100.000000


In [ ]:
df[(df['mean'] == 0) | (df['partial_mean'] == 0)]

,word,pystemmer,wordsplitting,part_res,res,partial_mean,mean
எங்களால்,எங்களால்,,[],{'': 0},{'': 0},0.0,0.0
கொண்டிருக்கிறான்,கொண்டிருக்கிறான்,,[],{'': 0},{'': 0},0.0,0.0
கொண்டிருக்கின்றன,கொண்டிருக்கின்றன,,[],{'': 0},{'': 0},0.0,0.0
உங்களுக்கும்,உங்களுக்கும்,,[],{'': 0},{'': 0},0.0,0.0
கொண்டிருக்கிறோம்,கொண்டிருக்கிறோம்,,[],{'': 0},{'': 0},0.0,0.0
கள்ளும்,கள்ளும்,,[],{'': 0},{'': 0},0.0,0.0
எங்களில்,எங்களில்,,[],{'': 0},{'': 0},0.0,0.0
பட்டுத்,பட்டுத்,,[],{'': 0},{'': 0},0.0,0.0
கொண்டிருக்கிறாள்,கொண்டிருக்கிறாள்,,[],{'': 0},{'': 0},0.0,0.0
பற்றிக்,பற்றிக்,,[],{'': 0},{'': 0},0.0,0.0


In [ ]:
stemmer.stemWord("எங்களால்")

''

In [ ]:
def hasQuestionSuffix(word):
  return (word[-1] in ["ா", "ே", "ோ"])

def fixQuestionSuffix(word):
  return word[:-1] + "்" 

def hasConjuctionSuffix(word):
  return (word[-3:] == "ும்")

def fixConjuctionSuffix(word):
  return word[:-3] + "்" 
  #return word[:-2]

def hasPluralSuffix(word):
  return (word[-5:] == "ங்கள்") or (word[-3:] == "கள்") or word[-4:] == "றனர்" or word[-3:] == "னர்" 

def fixPluralSuffix(word):
  if word[-5:] == "ங்கள்":
    return word[:-5] + "ம்"
  elif word[-3:] == "கள்":
    return word[:-3]
  elif word[-4:] == "றனர்":
    return (word[:-3]+ "ு")
  elif word[-3:] == "னர்":
    return word[:-3]

#implement from here

def hasImperativeSuffix(word):
   return (word[-1] in ["ி"])

def fixImperativeSuffix(word):
  #logic changed here
  #return word[:-1]
  return word[:-2]

def hasTenseSuffix(word):
    return (word[-6:] == "கின்றன") or (word[-3:] in ["ேன்"]) or (word[-16:-6]=="கொண்டிருக்")

def fixTenseSuffix(word):
  if (word[-4:]=="வேன்"):  #for future tense
     return word[:-4]
  if (word[-3:] in ["ேன்"]):
    return word[:-5] + "ு"
  if (word[-6:] == "கின்றன"):  # present
     return word[:-6]
  if(word[-16:-6]=="கொண்டிருக்"):
     return word[:-18]
  

def hasCommonWordSuffix(word):
  return  "ில்லாத" in word 

def fixCommonWordSuffix(word):
  return word[:-6]+ "்"

#வேற்றுமை உருப்புகள்
def hasCaseSuffix(word):
  return (word[-4:-2] == word[-2:]) or (word[-6:] in ["த்தில்"]) or (word[-4:] in ["ிடம்", "த்த்"]) or (word[-1]=='ை') or (word[-2:] in ["த்","ந்"]) or  (word[-4:] in ["த்து","ந்து","ந்த"] or word[-3:] in ["ந்த"]) or (word[-3:] == "ின்") or (word[-1:]== "ு")

#வேற்றுமை உருப்புகள்
def fixCaseSuffix(word):
  if(word[-4:-2] == word[-2:]):
    return word[:-2]
  if(word[-1:]== "ு"):
    if(word[-2]==word[-3]):
        return word[:-3]
    else:
        return word[:-2]
  if(word[-6:] in ["த்தில்"]):
    return word[:-6]+"ம்"
  if (word[-4:] in ["த்து","ந்து","ந்த"] or word[-3:] in ["ந்த"]):
    if(word[-3:] in ["ந்த"]):
      return word[:-3]
    return word[:-4]
  if word[-4:] == "த்த்":
    return word[:-4] + "ம்"
  if(word[-4:] == "ிடம்"):
    return word[:-4]+'்'
  if(word[-1]=='ை'):
    return word[:-1]+'்'
  if(word[-2:] in ["த்","ந்"]):  # to remove last extra mei eluthu
    return word[:-2]
  if word[-3:] == "ின்":
    return word[:-3] + '்'
  return word

In [ ]:
def stem(word):

  iteration = 0
  temp = word.replace('ஂ', '்')
  
  while temp and len(temp) > 4:

    iteration += 1
    # print("Beginning of iteration {} : {}".format(iteration, temp))

    if hasQuestionSuffix(temp):
      tempStore = temp
      temp = fixQuestionSuffix(temp)
      if(tempStore == temp):
        break

    elif hasConjuctionSuffix(temp):
      tempStore = temp
      temp = fixConjuctionSuffix(temp)
      if(tempStore == temp):
        break

    elif hasCaseSuffix(temp):
      tempStore = temp
      temp = fixCaseSuffix(temp)
      if(tempStore == temp):
        break

    elif hasPluralSuffix(temp):
      tempStore = temp
      temp = fixPluralSuffix(temp)
      if(tempStore == temp):
        break

    elif hasImperativeSuffix(temp):
      tempStore = temp
      temp = fixImperativeSuffix(temp)
      if(tempStore == temp):
        break

    elif hasTenseSuffix(temp):
      tempStore = temp
      temp = fixTenseSuffix(temp)
      if(tempStore == temp):
        break
      

    elif hasCommonWordSuffix(temp):
      tempStore = temp
      temp = fixCommonWordSuffix(temp)
      if(tempStore == temp):
        break

    else:
      break
    
  return temp

In [ ]:
stem('எங்களால்')

'எங்களால்'